<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-21 16:26:18
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.37 C
-------------------
Today PnL: -27.80 K (-0.2%)
Current PnL: -13.72 L (-9.56%)
CY Booked + Current PnL: -5.64 L (-3.93%)
-------------------
Total profit:  3.95 L
Total loss:  -17.67 L
-------------------
Total Booked + Current PnL: 21.34 L (18.31%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (5.88%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.97 L (52.36%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-224.30,H-SC,9.54,89700.0,-5616.0,5651.0,-2.12,-5.89,6.30,0.04,161.0,-0.99,0.65,76.92,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-9.61,M-LC,4.44,218413.0,16934.0,10287.0,0.10,8.40,4.71,13.51,58.0,1.65,1.59,22.06,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-20.73,H-LC,4.94,109232.0,4920.0,12616.0,1.01,4.72,11.55,16.81,9.0,0.39,0.80,21.52,X5K,ATH,METALS
17,BRITANNIA,5190.70,6446.05,13.95,H-LC,9.89,102267.0,8834.0,13765.0,-0.94,9.46,13.46,24.18,32.0,0.64,0.74,24.18,XY25,ATH,FMCG
86,UJJIVANSFB,52.77,53.00,49.36,M-SC,24.75,129222.0,-13257.0,13878.0,-2.07,-9.30,10.74,0.44,249.0,-0.96,0.94,52.52,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.0,-19.56,M-SC,2.05,167627.0,-148.0,55987.0,-1.41,-0.09,33.4,33.28,209.0,-0.00,1.22,29.78,XY24,NTT,FOOTWEAR
14,BERGEPAINT,561.33,680.0,-18.91,H-MC,11.18,229493.0,2154.0,45899.0,-1.76,0.95,20.0,21.14,108.0,0.05,1.67,28.49,XY24,NTT,PAINTS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-23.18,H-LC,0.17,201924.0,-2576.0,43999.0,-0.57,-1.26,21.79,20.25,11.0,-0.06,1.47,6.44,X40N,BTT,PHARMA
40,HINDUNILVR,2413.81,2723.00,-11.85,H-LC,1.62,276069.0,5722.0,28904.0,-0.99,2.12,10.47,12.81,16.0,0.20,2.01,15.46,XY25,NTT,FMCG
10,BAJAJHFL,122.26,152.00,-4.05,H-LC,2.00,199080.0,-1426.0,50208.0,-0.41,-0.71,25.22,24.33,23.0,-0.03,1.45,14.80,X40N,BTT,FINANCE
37,HCLTECH,1647.60,1943.91,5.38,H-LC,2.33,93354.0,-7150.0,25224.0,-1.19,-7.11,27.02,17.98,10.0,-0.28,0.68,13.54,X40,ATH,IT
41,HINDZINC,471.44,730.22,18.37,H-LC,3.12,193423.0,-11653.0,124216.0,2.10,-5.68,64.22,54.89,3.0,-0.09,1.41,17.04,X5K,ATH,METALS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,2.46,20100.0,-29493.0,693.0,5.24,-59.47,3.45,-58.07,201.0,-42.56,0.15,34.79,XY24,ATH,MISC
59,NATIONALUM,189.63,247.44,-48.64,H-MC,4.67,103345.0,2841.0,27800.0,3.71,2.83,26.90,30.49,82.0,0.10,0.75,36.15,MH,ATH,METALS
1,ABB,5551.76,7934.00,-36.27,H-LC,6.72,227604.0,11085.0,81824.0,3.36,5.12,35.95,42.91,7.0,0.14,1.66,18.97,AR,NTT,ELECTRICAL
79,TANLA,917.30,1963.11,-28.08,H-SC,5.30,176123.0,-63292.0,336254.0,2.86,-26.44,190.92,114.01,127.0,-0.19,1.28,61.53,AR,ATH,IT
28,DIGITIDE,188.38,255.97,NaN,NaN,0.48,56057.0,14802.0,0.0,2.79,35.88,0.00,35.88,197.0,inf,0.41,31.98,XY24,ATH,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,DEN,52.79,75.0,125.39,L-SC,5.40,63040.0,-21424.0,56963.0,-3.00,-25.36,90.36,42.07,265.0,-0.38,0.46,33.88,AR,NTT,ENTERTAINMENT
24,COFFEEDAY,59.14,80.0,-49.70,L-SC,21.48,73555.0,-39994.0,80043.0,-2.94,-35.22,108.82,35.27,266.0,-0.50,0.54,80.03,XR,NTT,HOTELS
69,SATIN,199.77,274.0,-21.16,H-SC,4.32,156371.0,-39603.0,112415.0,-2.84,-20.21,71.89,37.16,131.0,-0.35,1.14,16.78,XY24,NTT,FINANCE
89,VAIBHAVGBL,350.21,521.0,55.92,H-SC,7.19,104012.0,-47979.0,122100.0,-2.73,-31.57,117.39,48.77,124.0,-0.39,0.76,20.23,XR,NTT,JEWELLERY
87,UNIONBANK,123.87,163.0,-7.55,M-LC,18.66,162182.0,21342.0,23143.0,-2.61,15.15,14.27,31.59,71.0,0.92,1.18,40.88,XY24,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-224.30,H-SC,9.54,89700.0,-5616.0,5651.0,-2.12,-5.89,6.30,0.04,161.0,-0.99,0.65,76.92,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,49.36,M-SC,24.75,129222.0,-13257.0,13878.0,-2.07,-9.30,10.74,0.44,249.0,-0.96,0.94,52.52,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-28.99,M-SC,11.81,151528.0,-19613.0,19562.0,-0.31,-11.46,12.91,-0.03,194.0,-1.00,1.10,9.65,OX40N,NTT,DURABLES
21,CERA,7989.07,8422.0,-13.13,H-SC,2.61,124326.0,-19477.0,27265.0,-0.21,-13.54,21.93,5.42,154.0,-0.71,0.91,37.20,OX40N,NTT,CERAMICS
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,9.32,148825.0,-25734.0,25762.0,-0.07,-14.74,17.31,0.02,129.0,-1.00,1.08,29.05,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VBL,492.64,672.28,-13.32,H-LC,14.32,263243.0,-1797.0,98453.0,0.92,-0.68,37.40,36.46,1.0,-0.02,1.92,12.42,X40N,ATH,FMCG
32,ENRIN,1377.95,3176.70,NaN,NaN,1.44,76448.0,41999.0,2974.0,0.67,121.92,3.89,130.54,2.0,14.12,0.56,16.90,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,18.37,H-LC,3.12,193423.0,-11653.0,124216.0,2.10,-5.68,64.22,54.89,3.0,-0.09,1.41,17.04,X5K,ATH,METALS
82,TCS,3794.03,4998.00,-24.49,H-LC,11.12,246340.0,-49594.0,143493.0,-0.99,-16.76,58.25,31.73,4.0,-0.35,1.79,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-11.70,H-LC,7.22,283590.0,21989.0,123645.0,-0.11,8.41,43.60,55.67,6.0,0.18,2.06,15.02,X40,BTT,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,255.97,NaN,NaN,0.48,56057.0,14802.0,0.0,2.79,35.88,0.00,35.88,197.0,inf,0.41,31.98,XY24,ATH,IT
32,ENRIN,1377.95,3176.70,NaN,NaN,1.44,76448.0,41999.0,2974.0,0.67,121.92,3.89,130.54,2.0,14.12,0.56,16.90,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,96.87,M-MC,12.58,235811.0,10849.0,156814.0,0.36,4.82,66.50,74.53,193.0,0.07,1.72,36.91,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.82,34.69,H-SC,11.25,137768.0,12471.0,46648.0,-0.18,9.95,33.86,47.19,144.0,0.27,1.00,72.36,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,17.74,H-SC,7.30,150846.0,24646.0,21526.0,-1.43,19.53,14.27,36.59,151.0,1.14,1.10,34.49,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,5.87,H-SC,2.67,118936.0,16932.0,27415.0,0.86,16.60,23.05,43.48,121.0,0.62,0.87,36.16,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-2.97,H-SC,11.77,181960.0,17290.0,50676.0,-1.39,10.50,27.85,41.28,119.0,0.34,1.32,19.32,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,1.44,76448.0,41999.0,2974.0,0.67,121.92,3.89,130.54,2.0,14.12,0.56,16.90,AR,ATH,ELECTRICAL
53,KPIGREEN,497.21,731.82,34.69,H-SC,11.25,137768.0,12471.0,46648.0,-0.18,9.95,33.86,47.19,144.0,0.27,1.00,72.36,MH,ATH,POWER
94,WHIRLPOOL,1219.98,2270.00,-36.16,M-SC,11.09,107438.0,15940.0,62808.0,-0.84,17.42,58.46,86.07,207.0,0.25,0.78,55.42,XR,NTT,DURABLES
57,MASFIN,326.60,399.50,-10.57,H-SC,15.78,101550.0,3570.0,18299.0,-0.19,3.64,18.02,22.32,148.0,0.20,0.74,47.28,XR,ATH,FINANCE
45,INDIAMART,2327.09,4911.36,-47.14,H-SC,11.59,137811.0,14475.0,122486.0,-1.98,11.74,88.88,111.05,118.0,0.12,1.00,35.03,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,2.46,20100.0,-29493.0,693.0,5.24,-59.47,3.45,-58.07,201.0,-42.56,0.15,34.79,XY24,ATH,MISC
28,DIGITIDE,188.38,255.97,NaN,NaN,0.48,56057.0,14802.0,0.0,2.79,35.88,0.00,35.88,197.0,inf,0.41,31.98,XY24,ATH,IT
74,SIS,477.00,477.00,2289.12,M-SC,4.96,64277.0,-16813.0,16815.0,-0.83,-20.73,26.16,0.00,236.0,-1.00,0.47,30.27,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.48,73555.0,-39994.0,80043.0,-2.94,-35.22,108.82,35.27,266.0,-0.50,0.54,80.03,XR,NTT,HOTELS
84,TRIDENT,37.35,48.00,23.89,M-SC,4.56,77904.0,-14350.0,40658.0,0.19,-15.56,52.19,28.51,239.0,-0.35,0.57,33.31,XR,NTT,TEXTILES


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,2.46,20100.0,-29493.0,693.0,5.24,-59.47,3.45,-58.07,201.0,-42.56,0.15,34.79,XY24,ATH,MISC
64,RELAXO,902.64,1176.00,-29.51,H-SC,17.37,63712.0,-51826.0,86814.0,-2.05,-44.86,136.26,30.28,132.0,-0.60,0.46,24.47,X40N,NTT,FOOTWEAR
32,ENRIN,1377.95,3176.70,NaN,NaN,1.44,76448.0,41999.0,2974.0,0.67,121.92,3.89,130.54,2.0,14.12,0.56,16.90,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-10.57,H-SC,15.78,101550.0,3570.0,18299.0,-0.19,3.64,18.02,22.32,148.0,0.20,0.74,47.28,XR,ATH,FINANCE
53,KPIGREEN,497.21,731.82,34.69,H-SC,11.25,137768.0,12471.0,46648.0,-0.18,9.95,33.86,47.19,144.0,0.27,1.00,72.36,MH,ATH,POWER


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.51
1,25,41.00
2,50,69.97


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.81
LC    34.95
MC    22.09
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.78
X40N     14.34
X40      12.44
XY25     10.96
XR        9.29
OX40N     8.43
AR        8.27
X5K       2.21
MH        1.75
X200      1.35
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.14
H-SC    25.45
H-MC    18.96
M-SC    14.90
M-LC     5.77
M-MC     2.77
L-SC     1.46
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.15,-0.74,34.44
IT,11.78,-12.21,67.37
FINANCE,8.51,-11.49,51.95
BANKS,7.69,-7.99,52.06
MISC,5.94,-17.97,73.45
PAINTS,5.55,-14.45,37.01
ELECTRICAL,5.42,0.89,46.07
HEALTHCARE,4.35,-1.00,27.83
AUTO,4.34,-13.23,63.02


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2622639.0,25
XR,1018759.0,13
AR,1014246.0,10
X40,743383.0,10
X40N,580959.0,12
XY25,432971.0,8
OX40N,312436.0,11
SR,185807.0,2
X5K,136832.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2326444.0,26
M-SC,1580741.0,20
H-LC,1317898.0,21
H-MC,1102323.0,15
M-MC,297525.0,2
M-LC,269461.0,4
L-SC,199128.0,3
L-MC,53646.0,1
L-LC,46384.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,908338.0,7
M-SC,XY24,775946.0,7
H-SC,AR,513041.0,3
H-LC,X40,477135.0,6
H-MC,XY24,431110.0,4
H-SC,XR,336328.0,4
H-LC,X40N,302451.0,6
M-MC,XY24,297525.0,2
M-SC,XR,271787.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
